In [ ]:
import cv2
!pip install easyocr --q
!pip install requests --q
!pip install reportlab --q
import easyocr
import sqlite3
import requests
from reportlab.pdfgen import canvas
import os # import the os module


In [ ]:
def initialize_database():
    conn = sqlite3.connect("violations.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS violations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            plate_number TEXT,
            image_path TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    return conn
def send_email(email, pdf_path):
    return requests.post(
        "https://api.mailgun.net/v3/sandboxbf8fcb9ef5ae4626abbc2eb685875452.mailgun.org/messages",
        auth=("api", "e4dc8f18c4a657e9423afbbfb19723dd-ac3d5f74-a2bbded9"),
        files=[("attachment", ("e-challan.pdf", open(pdf_path, "rb")))],
        data={
            "from": "Traffic Authority <srijaknj9405@sandboxbf8fcb9ef5ae4626abbc2eb685875452.mailgun.org>",
            "to": email,
            "subject": "Traffic Violation Notice",
            "text": "You have violated the helmet rule. Find attached e-challan."
        }
    )
    print(f"Email sent to {email}, Status: {response.status_code}")

def process_frame(frame, helmet_cascade, plate_cascade, reader, db_conn):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    helmets = helmet_cascade.detectMultiScale(gray, 1.1, 4)

    if len(helmets) == 0:
        print("No helmet detected, searching for license plate...")
        plates = plate_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in plates:
            plate_crop = frame[y:y+h, x:x+w]
            plate_text = reader.readtext(plate_crop, detail=0)

            if plate_text:
                plate_number = plate_text[0]
                print(f"License Plate Detected: {plate_number}")
                # Save the snapshot with a unique name to prevent overwriting
                image_path = f"snapshot_{plate_number}.jpg"
                cv2.imwrite(image_path, frame)

                cursor = db_conn.cursor()
                cursor.execute("INSERT INTO violations (plate_number, image_path) VALUES (?, ?)", (plate_number, image_path))
                db_conn.commit()
                pdf_path = generate_echallan(image_path, plate_number)  # Pass the unique name
                send_email("sasanksai720@gmail.com", pdf_path)
                print("E-challan generated and email sent.")
                return  # Send immediate mail and return before continuing detection

def generate_echallan(image_path, plate_number):
    pdf_path = "e-challan.pdf"
    c = canvas.Canvas(pdf_path)
    c.drawString(100, 750, f"Violation Notice for {plate_number}")
    # Check if the image exists before drawing it
    if os.path.exists(image_path): # Check if the image file exists
        c.drawImage(image_path, 100, 500, width=300, height=200)
    else:
        print(f"Warning: Image file not found: {image_path}")
    c.save()
    print(f"E-challan generated: {pdf_path}")
    return pdf_path

def main():
    helmet_cascade = cv2.CascadeClassifier("/content/haarcascade_helmet.xml")
    plate_cascade = cv2.CascadeClassifier("/content/indian_license_plate.xml")
    reader = easyocr.Reader(["en"])
    db_conn = initialize_database()
    #db_conn = sqlite3.connect("violations.db")
    cap = cv2.VideoCapture("/content/WhatsApp Video 2025-02-28 at 8.05.54 PM (1).mp4")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        process_frame(frame, helmet_cascade, plate_cascade, reader, db_conn)

    cap.release()
    db_conn.close()
    print("Processing complete.")

if __name__ == "__main__":
    main()


No helmet detected, searching for license plate...
License Plate Detected: KLOZCRZ00Z
E-challan generated: e-challan.pdf
E-challan generated and email sent.
No helmet detected, searching for license plate...
License Plate Detected: KL0zCR200Z
E-challan generated: e-challan.pdf
E-challan generated and email sent.
No helmet detected, searching for license plate...
License Plate Detected: KLOZCR2OOZ
E-challan generated: e-challan.pdf
E-challan generated and email sent.
No helmet detected, searching for license plate...
License Plate Detected: KLOZER200Z
E-challan generated: e-challan.pdf
E-challan generated and email sent.
No helmet detected, searching for license plate...
License Plate Detected: KLOZCRZ00Z
E-challan generated: e-challan.pdf
E-challan generated and email sent.
No helmet detected, searching for license plate...
License Plate Detected: KLOZCRZQ02
E-challan generated: e-challan.pdf
E-challan generated and email sent.
No helmet detected, searching for license plate...
License

KeyboardInterrupt: 